In [77]:
import numpy as np 
import pandas as pd 
import opstrat as op
import scipy.stats as st
from scipy.stats import norm
from datetime import datetime
from scipy.optimize import fsolve
from option_tool.AMBT import bt_american,am_bs,implied_vol

In [98]:
St = 151.03
r = 0.0425
T0=datetime(2023,3,3) 
TT=datetime(2023,3,13)
T1=datetime(2023,4,21) 
TD=datetime(2023,3,15) 
rate=0
port = pd.read_csv('problem2.csv')

In [79]:
def return_calculate(price,method):
    names=price.columns
    num=len(names)
    names_2=names[names!='Date']

    if num==len(names_2):
        print("")
    else:
        num=num-1

    pos=np.where(names!='Date')[0]
    row_num=price.shape[0]
    out=np.zeros([row_num-1,num])
    for i in range(num):
        temp=np.array(price.iloc[:,pos[i]])

        if method.upper()=='DISCRETE':
            out[:,i]=temp[1:]/temp[0:-1]-1
        elif method.upper()=='LOG':
            out[:,i]=np.log(temp[1:])-np.log(temp[0:-1])
        else:
            print('method must in Log or Discrete')

    out=pd.DataFrame(out)
    out.index=price.index[1:]
    out.columns=price.columns[pos]

    return out

In [80]:
AAPL = pd.read_csv('DailyPrices.csv')
AAPL = return_calculate(AAPL,method='Discrete')
AAPL =AAPL['AAPL']
AAPLV =AAPL.std()

In [81]:
ttm =(T1-T0).days/365 
K = port['Strike']
O_type = np.where((port['OptionType'] == 'Call'),True, np.where((port['OptionType'] == 'Put'),False,np.nan))
opt_price = port['CurrentPrice']
K,O_type,opt_price

(0     150.0
 1     150.0
 2     150.0
 3     150.0
 4     150.0
 5     160.0
 6     150.0
 7     140.0
 8       NaN
 9     150.0
 10    150.0
 11      NaN
 12    155.0
 13      NaN
 14    145.0
 Name: Strike, dtype: float64,
 array([ 1.,  0.,  1.,  0.,  1.,  1.,  0.,  0., nan,  1.,  0., nan,  1.,
        nan,  0.]),
 0       6.80
 1       4.85
 2       6.80
 3       4.85
 4       6.80
 5       2.21
 6       4.85
 7       1.84
 8     151.03
 9       6.80
 10      4.85
 11    151.03
 12      4.05
 13    151.03
 14      3.01
 Name: CurrentPrice, dtype: float64)

In [104]:
ivol=[]
delta=[]
for i in range(len(K)):
    if port['Type'][i] == 'Option':
        temiv=implied_vol(St,K[i],ttm,r,rate,opt_price[i],O_type[i])
        ivol.append(temiv)
        if port['OptionType'][i]=='Call':
            opt = 'c' 
        else:
            opt = 'p' 
ivol.insert(8,0)
ivol.insert(11,0)
ivol.insert(13,0)
ivol

[0.2649281037758111,
 0.26186227132436496,
 0.2649281037758111,
 0.26186227132436496,
 0.2649281037758111,
 0.2336101965351821,
 0.26186227132436496,
 0.28238433791042467,
 0,
 0.2649281037758111,
 0.26186227132436496,
 0,
 0.2447132724090443,
 0,
 0.26921400715881616]

In [113]:
nsamples = 50
sim_rets = st.norm(0, AAPLV).rvs((10, nsamples))
sim_prices = 151.03 * (1 + sim_rets).prod(axis=0) 
len(sim_prices)

50

In [114]:
PV=[]
Strategy = port['Portfolio'].unique()
ret=pd.DataFrame()
mean=[]
VaR=[]
ES=[]
for i in range(len(Strategy)):
    data = port[port['Portfolio']==Strategy[i]]
    pv = (data['CurrentPrice']*data['Holding']).sum()
    PV.append(pv)
PV,Strategy

([11.649999999999999,
  1.9500000000000002,
  4.59,
  3.01,
  151.03,
  6.8,
  4.85,
  146.98,
  154.04],
 array(['Straddle', 'SynLong', 'CallSpread', 'PutSpread', 'Stock', 'Call ',
        'Put ', 'CoveredCall', 'ProtectedPut'], dtype=object))

In [118]:
def cal_div_period(T0, Td, T1, N):
    return int((Td - T0).days / (T1 - T0).days * N) 
ttm =(T1-TT).days/365 
nperiods =50 
div=1 
DP = cal_div_period(TT, TD, T1, nperiods)

In [121]:
for i in range(len(K)):
    for j in range(len(sim_prices)):
        if ivol[i] == 0:
            tem_price = sim_prices
        else:
            tem_price = bt_american(O_type[i], j, K[i], ttm, r, rate,ivol[i], nperiods, dividends=[(DP, div)])
    result=[(i,j,tem_price)]       

[(14, 49, 96.97348240845142)]

In [ ]:
def Normal_VaR(x,alpha):

    mu = np.mean(x)
    sd = np.std(x)
    VaR_Normal = -norm.ppf(alpha, mu, sd)

    return VaR_Normal

In [ ]:
def Normal_ES(x,alpha):
    mu = np.mean(x)
    sd = np.std(x)
    ES_Normal = -mu + sd * norm.pdf(norm.ppf(alpha, 0, 1)) / alpha

    return ES_Normal